<p style="font-size: 28px;">Выпускной проект по телекоммуникациям Елфимов Арсений</p>

**ЗАДАНИЕ:**  
  
Оператор связи «ТелеДом» хочет бороться с оттоком клиентов. Для этого его сотрудники начнут предлагать промокоды и специальные условия всем, кто планирует отказаться от услуг связи. Чтобы заранее находить таких пользователей, «ТелеДому» нужна **модель, которая будет предсказывать, разорвёт ли абонент договор.** Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и услугах. Ваша задача — **обучить на этих данных модель для прогноза оттока клиентов.**
  

---

Итак, нам необходимо на имеющихся данных разработать и обучить модель, чтобы прогнозировать разорвет ли пользователь договор с компанией или нет, для того чтобы специалисты могли выходить на клиента с высоким риском ухода со специальными промо предложениями. Перед нами стоит **задача бинарной классификации.**

Я предлагаю такой **план** наших дальнейших действий:  
1. Загрузить и осмотреть визуально все имеющиеся данные.
2. Обьединить данные в один общий фрейм.
3. Предобработать данные(проверить на наличие пропусков, дубликатов, мусора, проверить типы данных, названия столбцов и тп)
4. Исследовать данные, выявить закономерности и скореллированость признаков, оценить необходимоть использования каждого признака для обучения.
5. Подготовить данные для обучения(выделить целевой признак, разделить данные на тренировочные и тестовые данные, закодировать и отмасштабировать категориальные и числовые признаки соответсвсенно)
6. Выбрать несколько наиболее подходящих моделей и обучить их, основная метрика оценки на трейне ROC-AUC(должна быть больше 0.85)
7. Выбрать самую лучшую модель и проверить ее на тестовых данных основная метрика оценки на тесте ROC-AUC(должна быть больше 0.85)
8. Написать выводы по работе и дать рекомендации для бизнеса
   


---

# Импорты и константы

In [1]:
import pandas as pd

In [2]:
RANDOM_STATE = 20226

# Загрузка данных

Данные состоят из нескольких файлов, полученных из разных источников:  
**contract_new.csv** — информация о договоре;  
**personal_new.csv**— персональные данные клиента;  
**internet_new.csv** — информация об интернет-услугах;  
**phone_new.csv** — информация об услугах телефонии.  


In [3]:
contract_new = pd.read_csv('/Users/ars/Jupiter/Выпускной проект/contract_new.csv')

In [4]:
contract_new.head(1)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04


In [5]:
contract_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


`customerID` — идентификатор абонента;  
`BeginDate` — дата начала действия договора;  
`EndDate` — дата окончания действия договора;  
`Type` — тип оплаты: раз в год-два или ежемесячно;  
`PaperlessBilling` — электронный расчётный лист;  
`PaymentMethod` — тип платежа;  
`MonthlyCharges` — расходы за месяц;   
`TotalCharges` — общие расходы абонента.  

In [6]:
internet_new = pd.read_csv('/Users/ars/Jupiter/Выпускной проект/internet_new.csv')

In [7]:
internet_new.head(1)

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No


In [8]:
internet_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


`customerID` — идентификатор пользователя;  
`InternetService` — тип подключения;  
`OnlineSecurity` — блокировка опасных сайтов;  
`OnlineBackup` — облачное хранилище файлов для резервного копирования данных;  
`DeviceProtection` — антивирус;  
`TechSupport` — выделенная линия технической поддержки;  
`StreamingTV` — стриминговое телевидение;  
`StreamingMovies` — каталог фильмов.  

In [9]:
personal_new = pd.read_csv('/Users/ars/Jupiter/Выпускной проект/personal_new.csv')

In [10]:
personal_new.head(1)

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No


In [11]:
personal_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


`customerID` — идентификатор пользователя;  
`gender` — пол;  
`SeniorCitizen` — является ли абонент пенсионером;  
`Partner`— есть ли у абонента супруг или супруга;  
`Dependents` — есть ли у абонента дети.  

In [12]:
phone_new = pd.read_csv('/Users/ars/Jupiter/Выпускной проект/phone_new.csv')

In [13]:
phone_new.head(1)

,customerID,MultipleLines
0,5575-GNVDE,No


In [14]:
phone_new.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


`stomerID` — идентификатор пользователя;  
`MultipleLines` — подключение телефона к нескольким линиям одновременно.  

<div style="border: 2px solid #4ecdc4; border-radius: 10px; padding: 15px; background-color: #f7f7f7;">
<p style="font-size: 20px;">Вывод по шагу:</p>
<p>Туть </p>
</div>

# Обьединение данных

Во всех таблицах есть ключ - `customerID`, по которому можно обьединить таблицы.

Для начала обьединю contract_new и personal_new, так как в них одинаковое количесво строк и есть все данные

In [15]:
data_client = pd.merge(contract_new, personal_new, on='customerID', how = 'left') #Данные из contract_new должны отстаться в любом случае!

In [16]:
data_client.head(1)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04,Female,0,Yes,No


In [17]:
data_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   SeniorCitizen     7043 non-null   int64  
 10  Partner           7043 non-null   object 
 11  Dependents        7043 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 660.4+ KB


Отлчино! Ни одна строка не потерялась, вся клиентская база на месте

In [18]:
data_client_internet = pd.merge(data_client, internet_new, on='customerID', how = 'left') # присоединяем только туда где есть данные в таблице internet_new, если записи нет оставляем NaN

In [19]:
data_client_internet.head(1)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04,Female,0,Yes,No,DSL,No,Yes,No,No,No,No


In [20]:
data_client_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   SeniorCitizen     7043 non-null   int64  
 10  Partner           7043 non-null   object 
 11  Dependents        7043 non-null   object 
 12  InternetService   5517 non-null   object 
 13  OnlineSecurity    5517 non-null   object 
 14  OnlineBackup      5517 non-null   object 
 15  DeviceProtection  5517 non-null   object 
 16  TechSupport       5517 non-null   object 


In [21]:
data = pd.merge(data_client_internet, phone_new, on='customerID', how = 'left')

In [22]:
data.head(1)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   SeniorCitizen     7043 non-null   int64  
 10  Partner           7043 non-null   object 
 11  Dependents        7043 non-null   object 
 12  InternetService   5517 non-null   object 
 13  OnlineSecurity    5517 non-null   object 
 14  OnlineBackup      5517 non-null   object 
 15  DeviceProtection  5517 non-null   object 
 16  TechSupport       5517 non-null   object 


<div style="border-left: 5px solid #ff9f43; padding: 10px 15px; background-color: #fff9e6; box-shadow: 2px 2px 8px #ccc;">
<strong>Совет:</strong> Используйте такие блоки для визуального выделения советов или примечаний.
</div>

<div style="border: 2px solid #4ecdc4; border-radius: 10px; padding: 15px; background-color: #f7f7f7;">
<p style="font-size: 20px;">Вывод по шагу:</p>
<p>Данные обьеденины, ни одна строчка данных не была утеряна, все приклеили туда, куда нужно было приклеить, можно приступать ка предобработке данных и анализу</p>
</div>

# Предобработка данных

## Названия столбцов

Первым делом хочу поменять названия столбцов и переписать их в правильной стилистике, для этого создам список с правильными навзаниями столбцов

In [24]:
data.columns.tolist()

['customerID',
 'BeginDate',
 'EndDate',
 'Type',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'MultipleLines']

Закину в нейросеть и попрошу вернуть список с правильным написанием 

In [25]:
correct_columns = ['customer_id',
 'begin_date',
 'end_date',
 'type',
 'paperless_billing',
 'payment_method',
 'monthly_charges',
 'total_charges',
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'internet_service',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'multiple_lines']

In [26]:
data.columns = correct_columns

In [27]:
data.head(5)

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,gender,senior_citizen,partner,dependents,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,multiple_lines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,2071.84,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,No,Month-to-month,Yes,Mailed check,53.85,226.17,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1960.6,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN
4,9237-HQITU,2019-09-01,No,Month-to-month,Yes,Electronic check,70.70,353.5,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No


Любо-дорого посмотреть! Теперь займусь типами данных

## Типы данных

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   begin_date         7043 non-null   object 
 2   end_date           7043 non-null   object 
 3   type               7043 non-null   object 
 4   paperless_billing  7043 non-null   object 
 5   payment_method     7043 non-null   object 
 6   monthly_charges    7043 non-null   float64
 7   total_charges      7043 non-null   object 
 8   gender             7043 non-null   object 
 9   senior_citizen     7043 non-null   int64  
 10  partner            7043 non-null   object 
 11  dependents         7043 non-null   object 
 12  internet_service   5517 non-null   object 
 13  online_security    5517 non-null   object 
 14  online_backup      5517 non-null   object 
 15  device_protection  5517 non-null   object 
 16  tech_support       5517 

### Даты 

In [29]:
data['begin_date'] = pd.to_datetime(data['begin_date'], format='%Y-%m-%d')

In [30]:
data['end_date'] = pd.to_datetime(data['end_date'], format='%Y-%m-%d',  errors='coerce')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           1101 non-null   datetime64[ns]
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float64       
 7   total_charges      7043 non-null   object        
 8   gender             7043 non-null   object        
 9   senior_citizen     7043 non-null   int64         
 10  partner            7043 non-null   object        
 11  dependents         7043 non-null   object        
 12  internet_service   5517 non-null   object        
 13  online_security    5517 non-null   object        
 14  online_b

Тут сразу проявились данные об окончании договора - каждый пропуск в этом столбце значит, что договор действующий!     Итого, 1101 клиент из 7043 имеющихся прекратил сотрудничесвто с компанией 

### Числа

In [32]:
data['monthly_charges'] = pd.to_numeric(data['monthly_charges'], errors='coerce')

In [33]:
data['total_charges'] = pd.to_numeric(data['total_charges'], errors='coerce')

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           1101 non-null   datetime64[ns]
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float64       
 7   total_charges      7032 non-null   float64       
 8   gender             7043 non-null   object        
 9   senior_citizen     7043 non-null   int64         
 10  partner            7043 non-null   object        
 11  dependents         7043 non-null   object        
 12  internet_service   5517 non-null   object        
 13  online_security    5517 non-null   object        
 14  online_b